In [1]:
def dist(k1,k2):
    q = getRational(k2-k1)
    a,b = q.numerator(),q.denominator()
    return sqrt(2*(1-gcd(a,b)^2/(a*b)))

def kern(k1,k2):
    q = getRational(k2-k1)
    a,b = q.numerator(),q.denominator()
    return gcd(a,b)^2/(a*b)

def getRational(k):
    alpha = 2**(1/12.0)
    x = RDF(alpha**k).n(50)
    return x.nearby_rational(max_error=0.01*x)

alpha = (2**(1/12.0))
qq=[RDF(alpha**k).n(50).nearby_rational(max_error=0.01*RDF(alpha**k).n(50)) for k in range(12)]

print(qq)
M = matrix([[kern(k1,k2) for k1 in range(88)] for k2 in range(88)])
#print(M.str())
#print(M.is_positive_definite())

Ch = M.cholesky().rows()



[1, 16/15, 9/8, 6/5, 5/4, 4/3, 17/12, 3/2, 8/5, 5/3, 16/9, 15/8]


In [2]:
def parseMidi(fp,part=0):
    import os
    from music21 import converter
    print(fp)
    score = converter.parse(fp,quantizePost=True)
    #print(score.elements[0].notesAndRests)
    #print([e.partAbbreviation for e in score.elements][0])
    from music21 import chord
    durs = []
    ll0 = []
    for p in score.elements[part].notesAndRests:
        #print(p)
        if type(p)==chord.Chord:
            pitches = [e.pitch.midi-21 for e in p]
            dur = float(p.duration.quarterLength)
            #print(pitches)
            ll0.append(pitches)
            durs.append(dur)
        elif (p.name=="rest"):
            pass #ll0.append(0)
        else:   
            pitches = [p.pitch.midi-21]
            dur = float(p.duration.quarterLength)
            ll0.append(pitches)
            durs.append(dur)
            #print(p.name,p.octave,p.duration.quarterLength)
    #print(dir(score)) 
    #print(ll0)
    #print(durs)
    return ll0,durs

In [3]:
def getSingularValuesFromMidi(fp,part=0):
    import numpy as np
    from scipy import linalg
    ll0,durs = parseMidi(fp,part)
    #print(ll0)
    if len(ll0)==0:
        return None
    #print(ll0)
    print(fp,len(ll0),ll0[0:10],len(durs),durs[0:10])
    Ml = np.array([mean([vector([float(durs[k])*float(x) for x in Ch[kk]]) for kk in ll0[k]]) for k in range(len(ll0))])
    #Ml = np.array([mean([vector([float(x) for x in Ch[kk]]) for kk in ll0[k]]) for k in range(len(ll0))])
    U,s,V = linalg.svd(Ml,full_matrices=False)
    return Ml,U,s,V

In [ ]:
import os

mpath = "./good_midis"

midis = []
fns = os.listdir(mpath)
for fn in fns:
    midis.append(os.path.join(mpath,fn))

print(midis)        

ds = []
for m in midis:
    usv0 = getSingularValuesFromMidi(m,0)
    usv1 = getSingularValuesFromMidi(m,1)
    if not (usv0 is None ):
        ds.append((m,(usv0,usv1)))

ds = dict(ds)
#print(ds)

In [181]:
durationslist = [[sum([(2**(n-i)) for i in range(d+1)]) for n in range(-8,3+1)] for d in range(1)]
notevalues = []
for i in range(len(durationslist)):
    notevalues.extend(durationslist[i])
    
notevalues = sorted(notevalues)    
print(notevalues)
print(len(notevalues))

def findNearestDuration(duration):
    return sorted([(abs(duration-nv),nv) for nv in notevalues])[0][1]

[1/256, 1/128, 1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1, 2, 4, 8]
12


In [196]:

def writePitches(k0,k1,pitches,durations):
    from midiutil import MIDIFile

    track    = 0
    channel  = 0
    time     = 0   # In beats
    duration = 1   # In beats
    tempo    = 82 # In BPM
    volume   = 116 # 0-127, as per the MIDI standard

    ni = 2
    MyMIDI = MIDIFile(ni,adjust_origin=False) # One track, defaults to format 1 (tempo track
                     # automatically created)
    MyMIDI.addTempo(track,time, tempo)


    for k in range(2):
        MyMIDI.addProgramChange(k,k,0,0)


    times = [0,0]
    for k in xrange(len(pitches)):    
        channel = 0
        track = 0
        for i in range(len(pitches[k])):
            #pitch = degrees[0]
            track = k 
            channel = k
            pitch = pitches[k][i]
            duration = findNearestDuration(durations[k][i]*1.0)
            #print(k,pitch,times[k],duration,100)
            if pitch != -1: #rest
                MyMIDI.addNote(k, channel, pitch, times[k] , duration, 100)#*(ni-k+10.0)/(ni+10.0))
            times[k] += duration*1.0    
       
    with open("./mix/mix_"+k0.split("/")[-1]+"_vs_"+k1.split("/")[-1]+".mid", "wb") as output_file:
        MyMIDI.writeFile(output_file)
    print("written")    

In [197]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
from scipy.linalg import inv,pinv2,sqrtm,expm,logm, block_diag

nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(Ch)

def findBestMatch(nbrs,new_row):
    distances,indices = nbrs.kneighbors([np.array(new_row)])
    return indices[0][0]

def geometricMeanMatrix(A,B):
    return np.dot(A,sqrtm(np.dot(inv(A),B)))

def mixMatrix(M0,M1):
    M01 = np.vstack((M0,M1))
    M10 = np.vstack((M1,M0))
    
    U01,S01,V01 = sc.linalg.svd(M01,full_matrices=True)
    U10,S10,V10 = sc.linalg.svd(M10,full_matrices=True)                
    
    sigma = np.zeros((U01.shape[0],V01.shape[0]))
    for i in range(min(U01.shape[0],V01.shape[0])):
        sigma[i, i] = np.sqrt(S01[i]*S10[i])

    V = geometricMeanMatrix(V01,V10)
    U = geometricMeanMatrix(U01,U10)

    return np.dot(np.dot(U,sigma),V)
    
    

def getPitchesDurationsForPart(k0,k1,ds,part):
    M0,U0,S0,V0 = ds[k0][part]
    M1,U1,S1,V1 = ds[k1][part]
    M01 = matrix(np.vstack((M0,M1)))
    print(matrix(M0).rank())
    print(matrix(M1).rank())
    print(M01.rank())
    new = matrix(mixMatrix(M0,M1)).rows()
    pitches = []
    durations = []
    for v in new:
        if v.norm()<0.00001:
            pitch,duration = -1,1/4
        else:    
            pitch,duration = findBestMatch(nbrs,v/v.norm())+21,v.norm()
        #duration = findNearestDuration(duration)
        pitches.append(pitch)
        durations.append(duration)
        #print(pitch,duration)
    return pitches,durations




In [ ]:
print(ds.keys())
for k0 in ds.keys():
    for k1 in ds.keys():
        if k0!=k1:
            print(k0)
            print(k1)
            pitches0, durations0 = getPitchesDurationsForPart(k0,k1,ds,0)
            pitches = [pitches0]
            durations = [durations0]
            writePitches(k0,k1,pitches,durations)

<type 'module'>
